<img src="slides/10.png">

In [1]:
import pandas as pd
import numpy as np
loan = pd.read_csv('data/loan_data.csv')

# 使用しない列を削除
df = loan.drop(['Unnamed: 0', 'status', 'loan_amnt', 'term', 'annual_inc', 'dti',
                  'revol_bal', 'revol_util', 'purpose', 'home_ownership',
                  'delinq_2yrs_zero', 'pub_rec_zero', 'open_acc', 'grade'], axis=1)

# ダミー変数化
df['outcome'] = df['outcome'].map(lambda x: 1 if x == 'default' else 0)
df = pd.get_dummies(data=df)

# 説明変数、目的変数を作成
y = 'outcome'
X = df.drop([y], axis=1)
Y = df[y]

### 交差検証法により予測精度を検証

```cross_val_score()``` にモデルとデータセットを渡すだけ

- ```cv=5``` 5分割で実行
- ```scoring='roc_auc'``` 精度はAUC<br>
指定できる精度指標は以下を参照<br>
https://scikit-learn.org/stable/modules/model_evaluation.html

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import cross_val_score

def classify(depth):
    # 決定木を作成
    model = DecisionTreeClassifier(criterion='entropy', max_depth=depth)

    # 交差検証の実行
    score = cross_val_score(model, X, Y, cv=5, scoring='roc_auc')
    print('depth={:2}: AUC={:.3f} (+/- {:.3f})'.format(depth, score.mean(), score.std()))

### 決定木の深さの数値を変更して実行

4 から 11 未満まで値を変更しながら、決定木を作成し、交差検証法でのAUCを確認

In [3]:
for d in range(4, 11, 1):
    classify(d)

depth= 4: AUC=0.667 (+/- 0.037)
depth= 5: AUC=0.670 (+/- 0.038)
depth= 6: AUC=0.672 (+/- 0.035)
depth= 7: AUC=0.672 (+/- 0.035)
depth= 8: AUC=0.669 (+/- 0.034)
depth= 9: AUC=0.665 (+/- 0.033)
depth=10: AUC=0.660 (+/- 0.032)


- 決定木の深さが 6, 7 の時にAUCは最大
- 複数に分割して実行した結果のため、かなり安定した検証結果が得られる
- 決定木の深さの値を変更する範囲は、試行錯誤して決める必要